# **XGBoost**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xgboost

In [3]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/NotAnnotated/Regular/AveragePooled/split_features_1s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [5]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [6]:
train_data

{'melspectrogram': array([[-0.01720456, -0.9333485 , -0.9391832 , ..., -0.82813597,
         -0.8620985 , -0.9794059 ],
        [-0.01720306, -0.9333132 , -0.93914014, ..., -0.8327555 ,
         -0.86637986, -0.97935015],
        [-0.01720168, -0.9332716 , -0.93908674, ..., -0.847576  ,
         -0.8784226 , -0.9896085 ],
        ...,
        [-0.01720948, -0.55143994, -0.52205884, ..., -0.87508094,
         -0.90587527, -0.9997576 ],
        [-0.0171332 , -0.55125844, -0.51631904, ..., -0.8850505 ,
         -0.9144034 , -0.99985427],
        [-0.01714398, -0.5486961 , -0.5132266 , ..., -0.88104194,
         -0.911118  , -1.        ]], dtype=float32),
 'mfcc': array([[-1.        , -0.02898505, -0.10768069, ...,  0.01827751,
          0.02722204,  0.04308926],
        [-1.        , -0.02902972, -0.1065027 , ...,  0.01701666,
          0.02487057,  0.04410085],
        [-1.        , -0.0125336 , -0.09841443, ...,  0.02036034,
          0.02327715,  0.04241924],
        ...,
        [-0.9

In [7]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [8]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [9]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [10]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [11]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [12]:
display(train_y.shape)
display(train_y[:15])

(42578,)

array([ 7, 16, 19,  0,  1,  7,  6,  3,  1,  1, 19,  7,  9,  7, 12],
      dtype=int32)

In [13]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(42578, 128)

array([-0.02185728, -0.3939447 , -0.41050395, -0.50029534, -0.56085765,
       -0.6310321 , -0.6548134 , -0.6388136 , -0.63982546, -0.6337072 ,
       -0.64285135, -0.65793073, -0.6480776 , -0.6594148 , -0.6366135 ,
       -0.654948  , -0.6432404 , -0.6395858 , -0.6798748 , -0.6723758 ,
       -0.66720843, -0.69844174, -0.7145976 , -0.7270461 , -0.7289734 ,
       -0.75110966, -0.7298497 , -0.696753  , -0.68113   , -0.683587  ,
       -0.6695822 , -0.66871876, -0.66008353, -0.6631871 , -0.65656364,
       -0.6460304 , -0.6575156 , -0.6447011 , -0.6472018 , -0.6569579 ,
       -0.66957086, -0.67637694, -0.67121893, -0.6779927 , -0.68991876,
       -0.6964    , -0.7139648 , -0.7335337 , -0.7551437 , -0.7620292 ,
       -0.76579344, -0.7464363 , -0.71748465, -0.6991841 , -0.6900108 ,
       -0.6999048 , -0.70721406, -0.7059485 , -0.6748798 , -0.66050416,
       -0.6367882 , -0.62208366, -0.6234393 , -0.6094999 , -0.595338  ,
       -0.602499  , -0.62069196, -0.66656387, -0.7080173 , -0.73

'mfcc'

(42578, 20)

...

(42578, 84)

array([-12.80637 , -10.178921, -15.586308, -16.283943, -17.871552,
       -15.207264, -15.745362, -16.976467, -19.377678, -20.634443,
       -19.823278, -16.28474 , -11.230347, -13.031438, -22.283443,
       -23.941479, -24.802055, -24.640802, -26.405006, -22.172436,
       -23.847929, -29.4433  , -32.76458 , -32.2666  , -29.44668 ,
       -31.624424, -37.031097, -38.127777, -39.327816, -38.604076,
       -38.72474 , -38.8844  , -37.452694, -36.85629 , -38.343513,
       -37.406395, -37.946125, -37.184864, -38.44911 , -39.839275,
       -38.192352, -39.987495, -38.41094 , -37.54097 , -38.557415,
       -38.12515 , -37.687935, -40.436245, -40.11137 , -41.43069 ,
       -43.12546 , -45.00774 , -46.521675, -45.079754, -42.41699 ,
       -41.021828, -39.632954, -39.205273, -38.933006, -37.991035,
       -38.70373 , -40.62161 , -40.877434, -43.137257, -46.641926,
       -46.76184 , -43.449017, -42.309822, -41.863094, -38.54685 ,
       -35.893127, -34.787994, -36.57433 , -41.203064, -46.978

'id'

(42578,)

'XC184429.mp3'

In [14]:
display(val_y.shape)
display(val_y[:15])

(13083,)

array([ 3,  5, 11, 11, 18, 12, 19, 18,  1,  2,  1,  4, 17,  0, 11],
      dtype=int32)

In [15]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(13083, 128)

array([-0.01719964, -0.9332614 , -0.9346287 , -0.92839473, -0.9182133 ,
       -0.91571504, -0.90023047, -0.9096828 , -0.9013315 , -0.8832822 ,
       -0.8897478 , -0.8778112 , -0.8611843 , -0.87501085, -0.86042863,
       -0.87427056, -0.8929696 , -0.8929555 , -0.89963925, -0.90093535,
       -0.90265375, -0.90664506, -0.9189995 , -0.91942257, -0.9140625 ,
       -0.9264467 , -0.93026924, -0.9284385 , -0.934406  , -0.92323446,
       -0.93527853, -0.9328583 , -0.9292407 , -0.9284236 , -0.9269353 ,
       -0.9171119 , -0.9047776 , -0.8865346 , -0.90121865, -0.90096414,
       -0.9121269 , -0.91248345, -0.893796  , -0.90334725, -0.914868  ,
       -0.9018331 , -0.8862433 , -0.8832799 , -0.8790057 , -0.87130845,
       -0.86195666, -0.8688272 , -0.8709505 , -0.85542107, -0.8491089 ,
       -0.8495483 , -0.8412429 , -0.84774375, -0.84960586, -0.85435015,
       -0.85175586, -0.85534483, -0.8512846 , -0.85755765, -0.8516211 ,
       -0.8539241 , -0.8570129 , -0.83969903, -0.8464547 , -0.84

'mfcc'

(13083, 20)

...

(13083, 84)

array([-35.882477, -38.292088, -37.4063  , -40.330284, -42.694447,
       -42.67396 , -44.58036 , -46.30381 , -47.487316, -47.476955,
       -48.25535 , -50.481647, -55.270306, -54.9184  , -54.514065,
       -53.466293, -53.302032, -54.178078, -56.668327, -56.69416 ,
       -56.8258  , -58.064236, -57.938488, -56.90294 , -56.90026 ,
       -56.48227 , -58.99698 , -58.85235 , -58.181664, -55.98642 ,
       -57.59114 , -58.67091 , -59.410954, -58.078316, -58.395626,
       -56.53549 , -56.802666, -57.817825, -57.55971 , -57.068478,
       -55.886612, -55.62043 , -56.85259 , -55.615112, -57.423187,
       -58.65168 , -58.25182 , -58.85849 , -58.846207, -59.66834 ,
       -60.100693, -60.817085, -61.510292, -61.84868 , -62.075718,
       -62.074112, -62.60141 , -62.69142 , -60.156475, -58.66417 ,
       -58.836452, -58.94244 , -59.174664, -58.008244, -57.496716,
       -56.099037, -56.071007, -54.518993, -54.06901 , -54.937103,
       -54.929108, -54.014652, -54.42614 , -54.037262, -54.998

'id'

(13083,)

'XC511677.mp3'

## **Random Forest Model**

Saving evaluation results

In [16]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [17]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [18]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)

training_features.shape

(42578, 244)

In [19]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)

validation_features.shape

(13083, 244)

Fit the model with training data

In [20]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [21]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3200336314301001


In [22]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.28      0.47      0.35       793
           1       0.47      0.50      0.48      1294
           2       0.16      0.39      0.23       617
           3       0.57      0.60      0.59       984
           4       0.04      0.03      0.04       229
           5       0.01      0.00      0.01       303
           6       0.28      0.18      0.22       275
           7       0.53      0.52      0.53       698
           8       0.17      0.52      0.25       176
           9       0.24      0.38      0.29       227
          10       0.26      0.16      0.20       455
          11       0.07      0.00      0.01      1660
          12       0.44      0.24      0.31      1261
          13       0.41      0.23      0.29       502
          14       0.04      0.09      0.06       156
          15       0.29      0.36      0.32       798
          16       0.70      0.58      0.64   

### **With MFCCs only**

In [23]:
training_features = train_features['mfcc']

training_features.shape

(42578, 20)

In [24]:
validation_features = val_features['mfcc']

validation_features.shape

(13083, 20)

In [25]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [26]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 0.999553760157828
Validation accuracy: 0.3132309103416648


In [27]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.27      0.55      0.36       793
           1       0.46      0.54      0.50      1294
           2       0.14      0.33      0.19       617
           3       0.64      0.71      0.67       984
           4       0.21      0.14      0.17       229
           5       0.05      0.01      0.02       303
           6       0.25      0.14      0.18       275
           7       0.37      0.42      0.39       698
           8       0.18      0.52      0.27       176
           9       0.20      0.37      0.26       227
          10       0.39      0.37      0.38       455
          11       0.34      0.01      0.02      1660
          12       0.25      0.24      0.24      1261
          13       0.56      0.32      0.41       502
          14       0.11      0.14      0.12       156
          15       0.17      0.31      0.22       798
          16       0.50      0.21      0.29   

### **With Mel-Spectrogram Only**

In [28]:
training_features = train_features['melspectrogram']

training_features.shape

(42578, 128)

In [29]:
validation_features = val_features['melspectrogram']

validation_features.shape

(13083, 128)

In [30]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [31]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 0.9999765136925173
Validation accuracy: 0.27998165558358173


In [32]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.28      0.43      0.34       793
           1       0.39      0.47      0.43      1294
           2       0.25      0.37      0.30       617
           3       0.42      0.46      0.44       984
           4       0.01      0.03      0.02       229
           5       0.13      0.03      0.05       303
           6       0.31      0.16      0.21       275
           7       0.43      0.43      0.43       698
           8       0.15      0.53      0.23       176
           9       0.13      0.30      0.18       227
          10       0.38      0.17      0.23       455
          11       0.02      0.00      0.00      1660
          12       0.35      0.22      0.27      1261
          13       0.44      0.26      0.33       502
          14       0.02      0.04      0.03       156
          15       0.29      0.37      0.32       798
          16       0.60      0.40      0.48   

## **With Chroma Only**

In [33]:
training_features = train_features['chroma']

training_features.shape

(42578, 12)

In [34]:
validation_features = val_features['chroma']

validation_features.shape

(13083, 12)

In [35]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [36]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma'] = model.score(validation_features, val_y)

Training accuracy: 0.6862933909530744
Validation accuracy: 0.18107467706183597


In [37]:
val_scores['chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.11      0.43      0.17       793
           1       0.29      0.31      0.30      1294
           2       0.10      0.39      0.16       617
           3       0.50      0.33      0.40       984
           4       0.06      0.01      0.02       229
           5       0.34      0.12      0.18       303
           6       0.04      0.00      0.01       275
           7       0.14      0.21      0.17       698
           8       0.03      0.01      0.02       176
           9       0.16      0.13      0.14       227
          10       0.02      0.01      0.01       455
          11       0.13      0.01      0.01      1660
          12       0.19      0.08      0.11      1261
          13       0.14      0.04      0.06       502
          14       0.01      0.01      0.01       156
          15       0.28      0.49      0.36       798
          16       0.35      0.25      0.29   

## **With CQT Only**

In [38]:
training_features = train_features['cqt']

training_features.shape

(42578, 84)

In [39]:
validation_features = val_features['cqt']

validation_features.shape

(13083, 84)

In [40]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [41]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.22380188030268286


In [42]:
val_scores['cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.18      0.32      0.23       793
           1       0.39      0.47      0.43      1294
           2       0.07      0.10      0.08       617
           3       0.21      0.30      0.25       984
           4       0.07      0.03      0.04       229
           5       0.18      0.02      0.04       303
           6       0.08      0.07      0.07       275
           7       0.35      0.46      0.40       698
           8       0.19      0.45      0.27       176
           9       0.13      0.26      0.18       227
          10       0.12      0.07      0.09       455
          11       0.01      0.00      0.00      1660
          12       0.21      0.14      0.17      1261
          13       0.37      0.11      0.17       502
          14       0.04      0.17      0.07       156
          15       0.15      0.22      0.17       798
          16       0.57      0.57      0.57   

## **With MFCCs and Mel-Spectrogram**

In [43]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(42578, 148)

In [44]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(13083, 148)

In [45]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [46]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2864022013299702


In [47]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.27      0.50      0.35       793
           1       0.45      0.48      0.46      1294
           2       0.20      0.33      0.25       617
           3       0.55      0.55      0.55       984
           4       0.01      0.02      0.01       229
           5       0.00      0.00      0.00       303
           6       0.32      0.23      0.27       275
           7       0.44      0.46      0.45       698
           8       0.14      0.55      0.22       176
           9       0.12      0.36      0.18       227
          10       0.31      0.17      0.22       455
          11       0.13      0.00      0.01      1660
          12       0.47      0.23      0.31      1261
          13       0.54      0.40      0.46       502
          14       0.04      0.07      0.05       156
          15       0.27      0.29      0.28       798
          16       0.38      0.17      0.24   

## **With MFCCs and Chroma**

In [48]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(42578, 32)

In [49]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(13083, 32)

In [50]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [51]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 0.9998355958476208
Validation accuracy: 0.3337919437437897


In [52]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.28      0.56      0.37       793
           1       0.49      0.55      0.52      1294
           2       0.17      0.36      0.24       617
           3       0.64      0.74      0.69       984
           4       0.17      0.07      0.10       229
           5       0.06      0.02      0.03       303
           6       0.38      0.22      0.28       275
           7       0.40      0.40      0.40       698
           8       0.20      0.50      0.29       176
           9       0.20      0.37      0.26       227
          10       0.45      0.42      0.44       455
          11       0.32      0.01      0.02      1660
          12       0.31      0.25      0.27      1261
          13       0.60      0.35      0.44       502
          14       0.05      0.06      0.06       156
          15       0.18      0.43      0.25       798
          16       0.65      0.30      0.41   

## **With MFCCs and CQT**

In [53]:
training_features = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)

training_features.shape

(42578, 104)

In [54]:
validation_features = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)

validation_features.shape

(13083, 104)

In [55]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [56]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3040587021325384


In [57]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.27      0.45      0.34       793
           1       0.42      0.50      0.45      1294
           2       0.11      0.26      0.16       617
           3       0.60      0.57      0.58       984
           4       0.06      0.03      0.05       229
           5       0.00      0.00      0.00       303
           6       0.16      0.13      0.15       275
           7       0.42      0.48      0.45       698
           8       0.17      0.53      0.26       176
           9       0.28      0.35      0.31       227
          10       0.24      0.17      0.20       455
          11       0.06      0.00      0.00      1660
          12       0.41      0.28      0.34      1261
          13       0.59      0.30      0.40       502
          14       0.03      0.06      0.04       156
          15       0.19      0.33      0.24       798
          16       0.66      0.54      0.59   

## **With Chroma and CQT**

In [58]:
training_features = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)

training_features.shape

(42578, 96)

In [59]:
validation_features = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)

validation_features.shape

(13083, 96)

In [60]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [61]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.24558587479935795


In [62]:
val_scores['chroma_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.21      0.35      0.26       793
           1       0.44      0.48      0.46      1294
           2       0.15      0.24      0.18       617
           3       0.26      0.37      0.31       984
           4       0.08      0.04      0.06       229
           5       0.12      0.01      0.02       303
           6       0.13      0.10      0.11       275
           7       0.34      0.45      0.39       698
           8       0.18      0.41      0.25       176
           9       0.15      0.31      0.20       227
          10       0.13      0.07      0.09       455
          11       0.06      0.00      0.00      1660
          12       0.22      0.12      0.16      1261
          13       0.49      0.13      0.21       502
          14       0.04      0.19      0.07       156
          15       0.21      0.32      0.25       798
          16       0.58      0.58      0.58   

## **With Mel-Spectrogram and Chroma**

In [63]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(42578, 140)

In [64]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(13083, 140)

In [65]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [66]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = model.score(validation_features, val_y)

Training accuracy: 0.9999765136925173
Validation accuracy: 0.2919819613238554


In [67]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.32      0.45      0.37       793
           1       0.43      0.48      0.46      1294
           2       0.26      0.40      0.31       617
           3       0.41      0.46      0.44       984
           4       0.02      0.04      0.03       229
           5       0.05      0.01      0.02       303
           6       0.36      0.16      0.22       275
           7       0.43      0.43      0.43       698
           8       0.14      0.53      0.22       176
           9       0.11      0.36      0.17       227
          10       0.33      0.17      0.22       455
          11       0.01      0.00      0.00      1660
          12       0.45      0.29      0.35      1261
          13       0.54      0.25      0.34       502
          14       0.03      0.06      0.04       156
          15       0.34      0.51      0.40       798
          16       0.54      0.27      0.36   

## **With Mel-Spectrogram and CQT**

In [68]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)

training_features.shape

(42578, 212)

In [69]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)

validation_features.shape

(13083, 212)

In [70]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [71]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.28877168845066115


In [72]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.31      0.41      0.35       793
           1       0.37      0.50      0.43      1294
           2       0.26      0.44      0.33       617
           3       0.34      0.46      0.39       984
           4       0.03      0.03      0.03       229
           5       0.15      0.05      0.07       303
           6       0.05      0.03      0.04       275
           7       0.50      0.49      0.50       698
           8       0.17      0.51      0.26       176
           9       0.19      0.40      0.26       227
          10       0.27      0.10      0.15       455
          11       0.04      0.00      0.01      1660
          12       0.35      0.21      0.26      1261
          13       0.41      0.18      0.25       502
          14       0.03      0.06      0.04       156
          15       0.23      0.31      0.26       798
          16       0.74      0.60      0.66   

## **With Mel-Spectrogram, MFCCs, CQT**

In [73]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)

training_features.shape

(42578, 232)

In [74]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)

validation_features.shape

(13083, 232)

In [75]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [76]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.31002063746847053


In [77]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.28      0.47      0.35       793
           1       0.40      0.49      0.44      1294
           2       0.16      0.37      0.22       617
           3       0.56      0.58      0.57       984
           4       0.03      0.02      0.02       229
           5       0.01      0.00      0.01       303
           6       0.34      0.18      0.23       275
           7       0.52      0.50      0.51       698
           8       0.16      0.52      0.25       176
           9       0.29      0.41      0.34       227
          10       0.26      0.16      0.20       455
          11       0.09      0.00      0.01      1660
          12       0.47      0.29      0.36      1261
          13       0.43      0.25      0.31       502
          14       0.04      0.09      0.06       156
          15       0.22      0.25      0.23       798
          16       0.73      0.56      0.63   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [78]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(42578, 160)

In [79]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(13083, 160)

In [80]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [81]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3012306046013911


In [82]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.27      0.47      0.35       793
           1       0.45      0.47      0.46      1294
           2       0.21      0.37      0.27       617
           3       0.58      0.59      0.59       984
           4       0.01      0.03      0.02       229
           5       0.00      0.00      0.00       303
           6       0.41      0.21      0.27       275
           7       0.41      0.43      0.42       698
           8       0.15      0.57      0.24       176
           9       0.14      0.36      0.20       227
          10       0.31      0.17      0.22       455
          11       0.14      0.00      0.01      1660
          12       0.45      0.25      0.32      1261
          13       0.51      0.39      0.44       502
          14       0.04      0.08      0.05       156
          15       0.32      0.39      0.35       798
          16       0.50      0.26      0.34   

# Review results from all models

In [83]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.000,0.320
3,chroma,0.686,0.181
8,chroma_cqt,1.000,0.246
4,cqt,1.000,0.224
2,melspectrogram,1.000,0.280
9,melspectrogram_chroma,1.000,0.292
12,melspectrogram_chroma_mfcc,1.000,0.301
10,melspectrogram_cqt,1.000,0.289
11,melspectrogram_cqt_mfcc,1.000,0.310
1,mfcc,1.000,0.313


In [84]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features     0.308      0.802
3                    chroma     0.160      0.661
8                chroma_cqt     0.225      0.759
4                       cqt     0.205      0.751
2            melspectrogram     0.270      0.784
9     melspectrogram_chroma     0.281      0.799
10       melspectrogram_cqt     0.272      0.784
11  melspectrogram_cqt_mfcc     0.292      0.796
1                      mfcc     0.295      0.790
6               mfcc_chroma     0.315      0.806
7                  mfcc_cqt     0.292      0.814
5       mfcc_melspectrogram     0.277      0.785
